# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sys
from time import time
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
#nltk.download('averaged_perceptron_tagger')

import numpy as np
import pandas as pd

from sqlalchemy import create_engine
import pickle

from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.ensemble import AdaBoostClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, HashingVectorizer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import RandomizedSearchCV
import joblib

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ZoilaFlores\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ZoilaFlores\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ZoilaFlores\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# load cleaned data from SQL database
engine = create_engine('sqlite:///data/disaster_response.db')
df_clean = pd.read_sql_table('disaster_response_table',engine)

In [5]:
# observe SQL database connected and loaded properly
df_clean.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


In [6]:
#make copy of loaded dataframe
df = df_clean.copy()

In [7]:
# create message dataframe and target dataframe
X = df['message'].copy()
y = df.iloc[:,4:].copy()

In [8]:
# observe messages in "X"
X.head(2)

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
Name: message, dtype: object

In [9]:
# observe target categories are in "y"
y.head(2)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [8]:
# Tokenizer function to be called by main part of script
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    stopWords = set(stopwords.words('english'))

    # Get clean tokens after lemmatization, normalization, stripping and stop words removal
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        if tok not in stopWords:
            clean_tokens.append(clean_tok)
 
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
# basic setup for pipeline for model

def pipeline():
    pipeline = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
# use scikit to split the data into train and test 
# all model versions will use same split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [11]:
# create initial base model
model = pipeline() # create model using base pipeline
model.fit(X_train, y_train) # fit model based on split training data


In [15]:
t0 = time()
y_pred = model.predict(X_test) # predict model based on the reserved test data
print("done in %0.3fs" % (time() - t0))

done in 16.034s


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
# function to display results of models
def display_results(pred, test):
    # print general model accuracy at the top
    accuracy = (pred == test.values).mean()
    print('The model accuracy is {:.3f}'.format(accuracy))
    # basic scikit learn classification report by column
    for ind, cat in enumerate(test.keys()): 
        print("Classification report for {}".format(cat))
        print(classification_report(test.iloc[:,ind], pred[:,ind]))

    return

In [13]:
# display results from model 1
display_results(y_pred, y_test)

The model accuracy is 0.946
Classification report for related
              precision    recall  f1-score   support

           0       0.70      0.27      0.39      2054
           1       0.81      0.96      0.88      6534
           2       0.59      0.53      0.56        64

    accuracy                           0.79      8652
   macro avg       0.70      0.59      0.61      8652
weighted avg       0.78      0.79      0.76      8652

Classification report for request
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      7180
           1       0.88      0.43      0.58      1472

    accuracy                           0.89      8652
   macro avg       0.89      0.71      0.76      8652
weighted avg       0.89      0.89      0.88      8652

Classification report for offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8614
           1       0.00      0.00      0.00        38



E:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\ProgramData\Anaconda3\lib\site-packages\sklearn\metr

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      8548
           1       1.00      0.01      0.02       104

    accuracy                           0.99      8652
   macro avg       0.99      0.50      0.51      8652
weighted avg       0.99      0.99      0.98      8652

Classification report for refugees
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      8359
           1       0.20      0.00      0.01       293

    accuracy                           0.97      8652
   macro avg       0.58      0.50      0.49      8652
weighted avg       0.94      0.97      0.95      8652

Classification report for death
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      8246
           1       0.81      0.09      0.16       406

    accuracy                           0.96      8652
   macro avg       0.89      0.54      0.57      8652
weighte

E:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\ProgramData\Anaconda3\lib\site-packages\sklearn\metr

### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
# display possible model parameters to adjust
print(model.get_params().keys())

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf_

In [16]:
# select parameters for grid search #1

parameters = {
    'vect__min_df': [1, 5],
    'tfidf__use_idf':[True, False],
    'clf__estimator__n_estimators':[10, 25], 
    'clf__estimator__min_samples_split':[2, 5, 10]
}

cv = GridSearchCV(model, param_grid=parameters, n_jobs=-1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:
# test model with grid search for best parameters
t0 = time()
cv.fit(X_train, y_train)
print("done in %0.3fs" % (time() - t0))

GridSearchCV(estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             n_jobs=-1,
             param_grid={'clf__estimator__min_samples_split': [2, 5, 10],
                         'clf__estimator__n_estimators': [10, 25],
                         'tfidf__use_idf': [True, False],
                         'vect__min_df': [1, 5]})

In [18]:
t0 = time()
y_pred = cv.predict(X_test)
print("done in %0.3fs" % (time() - t0))

done in 3.060s


In [19]:
# display basic grid search results
display_results(y_pred, y_test)

The model accuracy is 0.948
Classification report for related
              precision    recall  f1-score   support

           0       0.69      0.26      0.38      2054
           1       0.80      0.96      0.88      6534
           2       0.54      0.55      0.54        64

    accuracy                           0.79      8652
   macro avg       0.68      0.59      0.60      8652
weighted avg       0.78      0.79      0.76      8652

Classification report for request
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      7180
           1       0.87      0.45      0.59      1472

    accuracy                           0.90      8652
   macro avg       0.88      0.72      0.77      8652
weighted avg       0.89      0.90      0.88      8652

Classification report for offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8614
           1       0.00      0.00      0.00        38



E:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.96      1.00      0.98      8245
           1       0.66      0.16      0.26       407

    accuracy                           0.96      8652
   macro avg       0.81      0.58      0.62      8652
weighted avg       0.95      0.96      0.94      8652

Classification report for buildings
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      8211
           1       0.83      0.17      0.28       441

    accuracy                           0.96      8652
   macro avg       0.90      0.58      0.63      8652
weighted avg       0.95      0.96      0.94      8652

Classification report for electricity
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      8467
           1       0.74      0.08      0.14       185

    accuracy                           0.98      8652
   macro avg       0.86      0.54      0.56      8652


E:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\ProgramData\Anaconda3\lib\site-packages\sklearn\metr

In [23]:
# tuned model grid search results
grid_scores = pd.DataFrame(cv.cv_results_)
print(grid_scores)

    mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0       32.969688      0.612943         0.973005        0.017475   
1       24.378272      0.504575         0.779204        0.005171   
2       32.078399      0.537681         0.936643        0.014023   
3       22.093608      0.263499         0.865715        0.128935   
4       82.050701      0.501355         2.196754        0.040297   
5       60.013235      0.980513         1.789771        0.137677   
6       80.681500      0.447480         2.145806        0.035919   
7       53.703887      0.693287         1.646118        0.063989   
8       25.099935      0.237622         0.963815        0.043183   
9       20.702031      0.104634         0.754029        0.010674   
10      24.231622      0.357736         0.922258        0.014997   
11      18.422562      0.122634         0.733451        0.008626   
12      60.571264      0.330734         2.096657        0.022099   
13      49.655026      0.400925         1.656107

In [24]:
# best parameters for tuned model grid search
print(cv.best_params_)

{'clf__estimator__min_samples_split': 10, 'clf__estimator__n_estimators': 25, 'tfidf__use_idf': True, 'vect__min_df': 5}


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [50]:
# pipeline for 2nd model using AdaBoostClassifier

def Pipeline2():
    
    pipeline2 = Pipeline([
        ('hashing', HashingVectorizer(tokenizer= tokenize)),
        ('clf', MultiOutputClassifier(AdaBoostClassifier(random_state = 30)))])
    
    
    return pipeline2

In [51]:
# 2nd model test 
t0 = time()
model2 = Pipeline2() # create model2 using base pipeline2
model2.fit(X_train, y_train) # fit model2 based on split training data
print("done in %0.3fs" % (time() - t0))

E:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


done in 1301.435s


In [52]:
# predict model based on the reserved test data
y_pred2 = model2.predict(X_test)
display_results(y_pred2, y_test)

The model accuracy is 0.946
Classification report for related
              precision    recall  f1-score   support

           0       0.51      0.10      0.16      2054
           1       0.77      0.97      0.86      6534
           2       0.43      0.14      0.21        64

    accuracy                           0.76      8652
   macro avg       0.57      0.40      0.41      8652
weighted avg       0.70      0.76      0.69      8652

Classification report for request
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      7180
           1       0.78      0.51      0.61      1472

    accuracy                           0.89      8652
   macro avg       0.84      0.74      0.78      8652
weighted avg       0.88      0.89      0.88      8652

Classification report for offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8614
           1       0.10      0.03      0.04        38



In [53]:
# list parameters for potential tuning
model2.get_params()

{'memory': None,
 'steps': [('hashing',
   HashingVectorizer(tokenizer=<function tokenize at 0x000002C03149D790>)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(random_state=30)))],
 'verbose': False,
 'hashing': HashingVectorizer(tokenizer=<function tokenize at 0x000002C03149D790>),
 'clf': MultiOutputClassifier(estimator=AdaBoostClassifier(random_state=30)),
 'hashing__alternate_sign': True,
 'hashing__analyzer': 'word',
 'hashing__binary': False,
 'hashing__decode_error': 'strict',
 'hashing__dtype': numpy.float64,
 'hashing__encoding': 'utf-8',
 'hashing__input': 'content',
 'hashing__lowercase': True,
 'hashing__n_features': 1048576,
 'hashing__ngram_range': (1, 1),
 'hashing__norm': 'l2',
 'hashing__preprocessor': None,
 'hashing__stop_words': None,
 'hashing__strip_accents': None,
 'hashing__token_pattern': '(?u)\\b\\w\\w+\\b',
 'hashing__tokenizer': <function __main__.tokenize(text)>,
 'clf__estimator__algorithm': 'SAMME.R',
 'clf__estimator__base_estimator':

In [60]:
# parameters for 2nd model test using grid search


parameters2 = {
    'clf__estimator__learning_rate': (0.6,1.0),
 
            }
 

In [63]:
# standard grid search - extremely time consuming, runs for days with above parameters2
t0 = time()
cv2 = GridSearchCV(model2, param_grid=parameters2, verbose=1)
print("done in %0.3fs" % (time() - t0))

done in 0.000s


In [64]:
# 2nd model grid search test
t0 = time()
cv2.fit(X_train, y_train)
print("done in %0.3fs" % (time() - t0))

Fitting 5 folds for each of 2 candidates, totalling 10 fits


E:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
E:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
E:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
E:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
E:\ProgramData\Anaconda3\lib\site-packages\s

done in 11180.581s


In [65]:
t0 = time()
y_pred2 = cv2.predict(X_test) # predict 2nd model
print("done in %0.3fs" % (time() - t0))

done in 12.686s


In [66]:
# display model 2 grid search results
display_results(y_pred2, y_test)

The model accuracy is 0.946
Classification report for related
              precision    recall  f1-score   support

           0       0.51      0.10      0.16      2054
           1       0.77      0.97      0.86      6534
           2       0.43      0.14      0.21        64

    accuracy                           0.76      8652
   macro avg       0.57      0.40      0.41      8652
weighted avg       0.70      0.76      0.69      8652

Classification report for request
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      7180
           1       0.78      0.51      0.61      1472

    accuracy                           0.89      8652
   macro avg       0.84      0.74      0.78      8652
weighted avg       0.88      0.89      0.88      8652

Classification report for offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8614
           1       0.10      0.03      0.04        38



In [67]:
# 2nd model grid search results
grid_scores2 = pd.DataFrame(cv2.cv_results_)
print(grid_scores2)

   mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0     968.408469     11.113283         7.437795        0.022539   
1     991.450670     35.865792         7.889636        0.796280   

  param_clf__estimator__learning_rate                                  params  \
0                                 0.6  {'clf__estimator__learning_rate': 0.6}   
1                                 1.0  {'clf__estimator__learning_rate': 1.0}   

   split0_test_score  split1_test_score  split2_test_score  split3_test_score  \
0           0.222317           0.211785           0.218047           0.214631   
1           0.226018           0.214347           0.226872           0.220325   

   split4_test_score  mean_test_score  std_test_score  rank_test_score  
0           0.220103         0.217377        0.003771                2  
1           0.203588         0.218230        0.008593                1  


In [68]:
# best parameters for 2nd model grid search
print(cv2.best_params_)

{'clf__estimator__learning_rate': 1.0}


In [ ]:
###### using random search to reduce number of iterations
rsearch = RandomizedSearchCV(estimator=model2, param_distributions=parameters2, n_jobs=-1, n_iter=2)
rsearch.fit(X_train, y_train)
print(rsearch)
# summarize the results of the random parameter search
print(rsearch.best_score_)
print(rsearch.best_estimator_.alpha)

### 9. Export your model as a pickle file

In [70]:
# Pickle best model
pickle.dump(cv.best_params_, open('data/disaster_model.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.